In [18]:
import numpy as np
import tensorflow as tf
import glob
from pydub import AudioSegment
from audioset import vggish_keras_embeddings
from time import time
import utils

start = time()

list_of_embeddings = []
audio_embedder = vggish_keras_embeddings.VGGishEmbedder(None)
file_path = 'The Speech that Made Obama President.webm'
for idx, val in enumerate(sound_slice_generator('The Speech that Made Obama President.webm', clipsize=1000, sample_rate=48000)):
    waveform, sample_rate = val[0], val[1]
    if val[0].shape[0] < sample_rate:
        waveform = np.pad(val[0], (0, sample_rate - val[0].shape[0]), mode='constant')
    processed_embedding =  audio_embedder.convert_waveform_to_embedding(waveform, sample_rate)

    embedding_final = np.expand_dims(processed_embedding, axis=0)
    list_of_embeddings.append((idx, embedding_final))
    if idx != 0 and idx % 100 == 0:
        print(idx, "samples embedded")

predictions = []

model = tf.keras.models.load_model('Models/cheer_applause_LSTM_ThreeLayer_100Epochs.h5')

for idx, v in list_of_embeddings:
    predictions.append((idx, model.predict(v)))
    
end = time()
print(end - start)

100 samples embedded
200 samples embedded
300 samples embedded
400 samples embedded
500 samples embedded
600 samples embedded
700 samples embedded


W0327 23:28:28.412578 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f003581a048>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 23:28:28.558465 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f003575b048>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 23:28:28.693015 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f00347edc50>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


45.019867181777954


In [26]:
import h5py, numpy as np


with h5py.File('The Speech that Made Obama President.webm.h5', 'w', libver='latest') as f:  # use 'latest' for performance

    for idx, v in list_of_embeddings:
        dset = f.create_dataset(str(idx), data=v, compression='gzip', compression_opts=9)

AttributeError: 'int' object has no attribute 'encode'

In [19]:
predictions = []

model = tf.keras.models.load_model('Models/cheer_applause_LSTM_ThreeLayer_100Epochs.h5')

for idx, v in list_of_embeddings:
    predictions.append((idx, model.predict(v)))

W0327 23:31:37.065338 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f00358d9748>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 23:31:37.200375 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7eff2037c3c8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0327 23:31:37.447947 139639421167424 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7eff20192470>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [22]:
for i in predictions:
    if i[1] > 0.59:
        m, s = divmod(i[0]/2, 60)
        print(int(m), 'min', s, 'sec', i[1][0])

1 min 31.0 sec [0.5960433]
1 min 32.0 sec [0.59046113]
2 min 18.5 sec [0.59937525]
3 min 37.5 sec [0.5910983]
3 min 53.0 sec [0.5969224]
3 min 53.5 sec [0.5981706]
4 min 9.0 sec [0.6120132]
4 min 29.0 sec [0.5920135]
4 min 29.5 sec [0.5994552]
4 min 30.0 sec [0.60140985]
4 min 30.5 sec [0.6043317]
5 min 11.5 sec [0.59660834]
5 min 14.5 sec [0.60013765]
5 min 19.0 sec [0.59182924]
5 min 20.0 sec [0.6055561]
5 min 48.5 sec [0.5936135]
5 min 49.5 sec [0.6193596]
5 min 50.0 sec [0.6200444]
5 min 50.5 sec [0.61896384]


In [14]:
def sound_slice_generator(sound_path, clipsize=500, sample_rate=48000):
    '''
    Generates np array value from sound clip to feed into VGGish Embedder

    input:
        list_sounds: generator of slices of sound clip
        clipsize:
    returns:
        generator to feed into vggish embedder
    '''
    sound = AudioSegment.from_file(sound_path)
    sound = sound.set_frame_rate(sample_rate)
    step = int(clipsize/2)
    list_sounds = sound[::step]  # generate clipsize/2 values of the clip
    prev_iter_value = None
    for idx, v in enumerate(list_sounds):

        if idx == 0:
            prev_iter_value = v
            continue
        overlapped = prev_iter_value + v # combine current 
        samples = overlapped.get_array_of_samples()
        np_samples = np.array(samples)
        s_reshaped = np_samples.reshape((-1,2))
        prev_iter_value = v
        #print(idx, s_reshaped.shape)
        yield s_reshaped, v.frame_rate

In [6]:
for idx, val in enumerate(sound_slice_generator('The Speech that Made Obama President.webm', 1000)):
    print(idx)

TypeError: slice indices must be integers or None or have an __index__ method

In [60]:
import pafy

In [121]:
#url = 'https://www.youtube.com/watch?v=IxAKFlpdcfc' # applause
#url = 'https://www.youtube.com/watch?v=EBGb40yh4SY' # denzel speech
#url = 'https://www.youtube.com/watch?v=408o4ImaHB8' # tennis match
#url = 'https://www.youtube.com/watch?v=vP4iY1TtS3s' # I have a dream speech
url = 'https://www.youtube.com/watch?v=OFPwDe22CoY' # obama speech

In [122]:
video = pafy.new(url)

In [123]:
audiostreams = video.audiostreams
for a in audiostreams:
    print(a.bitrate, a.extension, str(a.get_filesize()/(1024**2)))

50k webm 2.3237075805664062
70k webm 2.941255569458008
128k webm 4.132859230041504
128k m4a 5.646934509277344
160k webm 5.846334457397461


In [124]:
bestaudio = video.getbestaudio()
bestaudio.bitrate

'160k'

In [125]:
file = bestaudio.download()

In [5]:
from pydub import AudioSegment

In [6]:
song = AudioSegment.from_file('Applause.webm')

In [12]:
song.export('Applause.wav', format='wav')

<_io.BufferedRandom name='Applause.wav'>

In [7]:
song = AudioSegment.from_wav('Applause.wav')

In [8]:
len(song)

8009

In [10]:
song.frame_rate

48000

In [17]:
song_slices = song[::500]

In [36]:
from scipy.io import wavfile

sr, wav_data = wavfile.read('Applause.wav')
wav_data.shape

(384449, 2)

In [40]:
import array
import numpy as np
from pydub import AudioSegment

sound = AudioSegment.from_file('Applause.wav')
samples = sound.get_array_of_samples()

In [45]:
np.array(samples).reshape((-1,2)).shape

(384449, 2)

In [39]:
sr, wav_data = wavfile.read('Applause.wav')
wav_data[0]

array([0, 0], dtype=int16)

In [47]:
wav_data.shape

(384449, 2)

In [46]:
sr

48000

In [48]:
sound.frame_rate == sr

True